# 🏥 Affinage (*fine-tuning*) de modèles décodeurs (LLMs) PARTAGES pour la réponse aux questions à choix multiples

### Résumé du processus d'affinage

Une méthode courante pour entraîner efficacement de grands modèles pour des tâches en aval consiste à insérer des matrices entraînables plus petites qui sont une décomposition de bas rang (*Low-Rank Decomposition*) de la matrice de poids delta à apprendre pendant le fine-tuning.
Cette famille de techniques s'appelle donc **LoRA** (Low-Rank Adaptation).
Elles permettent de réduire considérablement le nombre de paramètres entraînables (*trainable parameters*) et ainsi le coût de calcul par rapport au fine-tuning complet où tout l'ensemble des paramètres sont modifiées pour adapter le modèle à la tâche.

Pour plus d'informations sur LoRA
- [Article fondamental](https://arxiv.org/abs/2106.09685)
- [Article d'analyse théorique](https://openreview.net/pdf?id=likXVjmh3E)
- [Documentation HF](https://huggingface.co/docs/peft/task_guides/lora_based_methods)

En pratique, ce notebook implémente ce processus d'adaptation en 6 étapes :
1. Chargement et préparation des données - on utilise ici un jeu de données de QCM médicales hébergé dans l'espace Hugging Face de l'équipe de développement de PARTAGES.
2. Définition des métriques à rapporter - une fonction adaptée au filtrage des réponses d'un modèle génératif spécifique à la tâche de réponses aux QCM.
3. Chargement du modèle de base et son tokeniseur.
4. Préparation de l'apprentissage - il faut ensuite définir les métaparamètres de LoRA ainsi que celles de l'optimisation de ses poids - taux d'apprentissage, taille de lot etc.
5. Lancement de l'apprentissage.
6. Évaluation du modèle entraîné (inférence).

### Installation des libraries

In [ ]:
# PyTorch
%pip install torch

# libraries Hugging Face
%pip install 'transformers[torch]' datasets peft trl accelerate

# autre
%pip install tensorboardX

In [ ]:
######################################
##########    À Modifier    ##########
######################################

# Si le jeu de donnée, modèle ou tokeniseur n'est pas public sur huggingface il
# faut utiliser un jeton de connexion à créer ici : https://huggingface.co/settings/tokens

# Pour l'instant les modèles PARTAGES sont privés et accessibles aux personnes munies d'une clé API
HF_TOKEN = ""

######################################

# Petit hack pour éviter un avertissement spécifique à Google Colab sur le
# chargement du jeton huggingface
try:
    from huggingface_hub.utils import _auth
    _auth._get_token_from_google_colab = lambda: None
except ImportError:
    pass

In [ ]:
################################
### modules utiles générales ###
################################
import os
import re
import json
from pathlib import Path
from typing import Dict, List, Union
from collections import defaultdict
from tqdm import tqdm

In [ ]:
HERE = os.path.dirname(os.getcwd())

### Chargement et préparation des données

Les jeux de données pour lesquels ce tutoriel est conçu, `PARTAGES-dev/frenchmedmcqa-sft` et `PARTAGES-dev/mediqal-sft` comprennent les caractéristiques suivantes :
- `doc_id` : identifiants unique pour chaque document
- `prompt` : l'entrée pour le modèle, structuré pour les *chat templates* des LLMs avec une liste de dictionnaires avec des attributs `content` et `role` ; ici nous avons
    - Le *system prompt* (`"role": "system"`) qui définit le contexte de la tâche - ce message et le même pour tous les exemples dans `frenchmedmcqa` avec une petite variation en `mediqal` pour préciser la spécialité médicale qui fait l'objet de la question.
    - La question à laquelle le modèle doit répondre (`"role": "user"`), avec 4-5 options différentes.
- `completion` : la sortie attendue ; dans ce contexte, uniquement des lettres désignant la ou les bonnes réponses.
- `word_count` : Le nombre de mots total qui sera passé au tokeniseur pour chaque question (y compris le system prompt).


In [ ]:
from datasets import load_dataset

#### Choisir un jeu de données

Nous avons préparé deux jeux de données composés de QCMs et formattés pour cette tâche.

`frenchmedmcqa-sft` a 2 171 questions dans le jeu de *train* et 312 dans le jeu d'évaluation, et `mediqal-sft` comprend 15 880 questions de train et 4 027 d'évaluation.

Modifiez la variable `DATASET_ID` pour choisir celui que vous allez utiliser.

In [ ]:
DATASET_ID = 'PARTAGES-dev/frenchmedmcqa-sft'
RANDOM_SEED = 42

In [ ]:
ds = load_dataset(DATASET_ID, token=HF_TOKEN)
print(ds)

In [ ]:
train_ds_init = ds["train"]
eval_ds = ds["validation"]

##### Jeu de développement

Nous allons utiliser un sous-ensemble du jeu de train pour suivre des métriques intrinséques pendant l'entraînement.

In [ ]:
dev_frac = .05  # À modifier si souhaité
dev_split_ds = train_ds_init.train_test_split(test_size=dev_frac, shuffle=True, seed=RANDOM_SEED)
train_ds = dev_split_ds["train"]
dev_ds = dev_split_ds["test"]

Regardons un exemple d'un document de ce jeu de données...

(vérifiez qu'il correspond au schéma décrit au dessus)

In [ ]:
example_instance = train_ds.shuffle()[0]  # échantilloner un exemple au hasard

for k, v in example_instance.items():  # c'est un dictionnaire
    if isinstance(v, list):  # prompt
        print(f"\n* {k} *")
        for turn_name, str_ in v[0].items():
            print(f"{turn_name} : {str_}")
        continue
    print(f"\n* {k} * : {v}", end="\n" * 2)

### Définition des métriques a rapporter

Nous allons évaluer les réponses du modèle de deux points de vue :
1. Comme des réponses à un examen, ou la réponse est, de manière globale, soit correcte soit fausse - d'où la métrique `exact_match`.
2. Comme des estimations pour un problème de classification - vu que nous avons un ensemble fixe de réponses valides possibles, nous pouvons considérer le problème comme une tâche de classification de documents multi-catégories. Nous calculons donc les métriques suivantes aussi, basées sur les éléments individuels (lettres A, B, etc.) de la réponse :
    - `precision` : parmi les réponses du modèle, quelle proportion était correcte ?  
    - `recall` : parmi les réponses correctes, combien ont été saisies par le modèle ?
    - `f1` : moyenne harmonique de `precision` et `recall`

In [ ]:
import torch
from datasets import Dataset
from transformers import PreTrainedModel, PreTrainedTokenizerFast
from peft import PeftModelForCausalLM

In [ ]:
def mcqa_eval(
    model: Union[PreTrainedModel, PeftModelForCausalLM],
    tokenizer: PreTrainedTokenizerFast,
    dataset: Dataset,
    batch_size: int,
    max_new_tokens: int,
    return_all_outputs: bool = False,  # pour déboguer
) -> Dict[str, float]:
    if not tokenizer.pad_token_id:
        tokenizer.add_special_tokens({"pad_token": tokenizer.eos_token})

    # compilation regex qui nous permettra de filter les sorties
    letters = dataset.map(
        lambda x: {'letters': re.sub(r'[^A-Z]', '', x['completion'][0]['content'])},
        desc="Extracting MCQ answer options from evaluation dataset",
    )['letters']
    answer_options = ''.join(set(''.join(letters)))
    mcq_answer_pattern = re.compile(fr'[,\.\s>][{answer_options}][,\.\s<]')

    # par souci d'efficacité, nous trions les séquences de tokens en fonction de leur longueurs
    # cela permet d'utiliser le moins de tokens de "padding" possible
    if not "seq_len" in dataset.features:
        dataset = dataset.map(
            lambda x: {"seq_len": len(x["input_ids"])},
            desc="Counting tokens",
        ).sort("seq_len", reverse=False)
    batched_dataset = dataset.batch(batch_size)

    if return_all_outputs:
        generations = {}
        num_batch_tokens = []
    total_docs = len(dataset)
    eval_counts = defaultdict(int)  # celui va stocker les éléments d'entrée pour les métriques
    generation_input_keys = "input_ids", "attention_mask"
    for batch_idx, batch in enumerate(tqdm(
        batched_dataset, desc="Génération et Évaluation",
    )):
        batch_max_length = max(batch["seq_len"])  # longueur de séquence maximale dans ce lot
        batch_model_input = {k: v for k, v in batch.items() if k in generation_input_keys}
        batch_tensors = tokenizer.pad(  # homogéniser les longueurs de séquence pour pouvoir passer une matrice au modèle
            batch_model_input, "longest", "left", return_tensors="pt"
        ).to(model.device)
        if return_all_outputs:
            batch_dim = batch_tensors["input_ids"].shape
            num_batch_tokens.append(batch_dim[0] * batch_dim[1])
        with torch.no_grad():
            # tourner le modèle
            outputs = model.generate(
                **batch_tensors,
                top_p=.9,
                temperature=.1,
                max_new_tokens=max_new_tokens,
                pad_token_id=tokenizer.pad_token_id
            )
        output_ids = outputs.detach().cpu().numpy()  # récuperer les tokens pour le CPU
        output_text_list = tokenizer.batch_decode(output_ids)  # transformer les tokens de sortie en mots
        for i, (id_, str_, label) in enumerate(
            zip(batch["doc_id"], output_text_list, batch["completion"])
        ):
            input_text = tokenizer.decode(batch_tensors["input_ids"][i])
            answer_split_idx = len(input_text)
            answer = str_[answer_split_idx:]
            answer_matches = re.findall(mcq_answer_pattern, answer)
            answer_set = set(map(lambda s: re.sub('[^A-Z]', '', s), answer_matches))
            target = label[0]["content"].strip("\n")  # vérité terrain - la réponse correcte
            y_set = set(target.split(","))
            if answer_set:
                overlap = answer_set.intersection(y_set)
                num_correct_responses = len(overlap)  # vraies positives
                num_incorrect_responses = len(answer_set - overlap)  # fausses positives
                num_missed_responses = len(y_set - overlap)  # fausses négatives
                exact_match = int(num_correct_responses == len(y_set))
            else:
                num_correct_responses = num_incorrect_responses = exact_match = 0
                num_missed_responses = len(y_set)
            eval_counts["num_correct_responses"] += num_correct_responses
            eval_counts["num_incorrect_responses"] += num_incorrect_responses
            eval_counts["num_missed_responses"] += num_missed_responses
            eval_counts["exact_match"] += exact_match
            if return_all_outputs:
                generations[id_] = str_
    metrics = {}
    metrics["accuracy"] = eval_counts["exact_match"] / len(dataset)
    precision_denom = eval_counts["num_correct_responses"] + eval_counts["num_incorrect_responses"]
    precision = (eval_counts["num_correct_responses"] / precision_denom) if precision_denom else 0.
    metrics["precision"] = precision
    recall_denom = eval_counts["num_correct_responses"] + eval_counts["num_missed_responses"]
    recall = (eval_counts["num_correct_responses"] / recall_denom) if recall_denom else 0.
    metrics["recall"] = recall
    f1_denom = precision + recall
    metrics["f1"] = (2 * precision * recall / f1_denom) if f1_denom else 0.
    ret = {"metrics": metrics}
    if return_all_outputs:
        ret["generations"] = generations
        ret["eval_counts"] = eval_counts
        ret["num_batch_tokens"] = num_batch_tokens

    return ret

### Chargement du tokeniseur et du modèle de base

In [ ]:
# Ici on sélectionne un des modèles du projet PARTAGES
# À modifier pour utiliser un autre modèle comme base d'affinage
PARTAGES_MODEL_ID = "PARTAGES-dev/Qwen3-4B-PDAPT-SLERP"
BASE_MODEL_NAME = "Qwen/Qwen3-4B-Base"

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
# Vu que nous avons adapté ce modèle à partir d'un modèle de fondation existant, le tokeniseur est
# identique à celui de ce modèle de base
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME, padding_side='left')
# tokenizer.add_special_tokens({'pad_token': ''})

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(PARTAGES_MODEL_ID, token=HF_TOKEN)
# base_model.resize_token_embeddings(len(tokenizer), mean_resizing=False)

### Préparation de l'entrainement

1. D'abord on fait la config de finetuning (LoraConfig)
2. Puis on initialise des adaptateurs LORA sur le modèle de base, ce qui nous donne une version specialisée (`PeftModelForCausalLM`) de l'architecture
3. Ensuite nous préparons les paramètres d'apprentissage plus générales

In [ ]:
from peft import LoraConfig, get_peft_model

In [ ]:
## Configuration LoRA
# cf. https://huggingface.co/docs/peft/v0.18.0/en/package_reference/lora#peft.LoraConfig
modules_to_save = ['lm_head', 'embed_tokens']
peft_config = LoraConfig(
    r=16,
    lora_alpha=10,
    lora_dropout=0.05,
    target_modules='all-linear',
    modules_to_save=modules_to_save,
    task_type='CAUSAL_LM',
    use_dora=True,
    init_lora_weights='pissa_niter_16',
    ensure_weight_tying=True
)

## Fusion du modèle de base avec les adaptateurs
peft_model = get_peft_model(base_model, peft_config)
print(peft_model)

In [ ]:
from trl import SFTConfig, SFTTrainer
from transformers import logging

In [ ]:
# création de fichier pour définir le processus de traitement de prompts par le modèle
# plus d'infos ici : [link]
# vous pouvez également utiliser un autre fichier .jinja si vous préférez, ou bien customiser celui-ci.
if hasattr(tokenizer, 'chat_template'):
    chat_template_path = Path(HERE) / 'chat-template-tmp.jinja'
    with chat_template_path.open('w') as f:
        f.write(tokenizer.chat_template)
    chat_template_path = str(chat_template_path)

# le template défini ci-dessus va s'occuper du rajout de certains tokens spéciaux nécessaire pour le bon
# traitement des instructions, donc ici on précise des arguments à passer au SFTTrainer qui indiquera que
# ces étapes ne sont pas nécessaire au niveau du traitement du jeu de données
dataset_kwargs = {'add_special_tokens': False, 'append_concat_token': False}

Les résultats de l'entraînement seront stocké à l'endroit précisé par `OUTPUT_DIR`.

L'API `Trainer` sauvegarde des points de contrôle (*checkpoints*) pendant l'entraînement.

In [ ]:
##############################################################################
######################### Paramétres d'apprentissage #########################
### À modifier selon vos environnements, expérimentation, cas d'usage etc. ###
##############################################################################

output_name = Path(BASE_MODEL_NAME).name + "-" + Path(DATASET_ID).name
OUTPUT_DIR = Path(os.getenv("HOME")) / output_name  # chemin vers le dossier qui stockera les résultats et les sorties
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
print("Dossier de sortie:", OUTPUT_DIR)

# paramètres qui gérent la durée de l'entraînement et l'utilisation de mémoire
# à modifier selon les ressources de calcul disponibles
EPOCHS = 4
BATCH_SIZE_TR = 8
BATCH_SIZE_EV = 8

# accumulation de gradients - permet d'ajuster la taille de lot effective (_effective batch size_)
# i.e. ici la taille de lot effective est BATCH_SIZE_TR * GRAD_ACC * NB_GPU
GRAD_ACC = 2

# définition des algorithmes à utiliser pour mettre à jour les poids du modèle (le cœur de l'apprentissage)
OPTIMIZER = 'adamw_torch_fused'
LR_VALUE = 2e-5
LR_SCHEDULE = 'linear'

# configuration de l'apprentissage
# cf. https://huggingface.co/docs/trl/v0.29.0/en/sft_trainer#trl.SFTConfig
training_args = SFTConfig(
    output_dir=OUTPUT_DIR / 'checkpoints',
    logging_dir=OUTPUT_DIR / 'logs',
    ####
    # num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE_TR,
    per_device_eval_batch_size=BATCH_SIZE_EV,
    torch_empty_cache_steps=4,
    ####
    gradient_accumulation_steps=GRAD_ACC,
    ####
    optim=OPTIMIZER,
    learning_rate=LR_VALUE,
    lr_scheduler_type=LR_SCHEDULE,
    warmup_ratio=.15,
    max_grad_norm=.3,
    completion_only_loss=True,
    ####
    chat_template_path=chat_template_path,
    dataset_kwargs=dataset_kwargs,
    ####
    eval_steps=.1,  # tourner l'évaluation sur `eval_ds` 10 fois pendant l'apprentissage
    logging_steps=.05,  #  faire le logging (progression générale de l'apprentissage) 20 fois
    save_strategy="epoch",  # sauvegarder un checkpoint après chaque époque
    ####
    bf16=False,  # décommente cette ligne en cas de 'ValueError: Your setup doesn't support bf16/gpu.'
    seed=RANDOM_SEED,
    push_to_hub=False,
)
trainer = SFTTrainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    processing_class=tokenizer,
)

In [ ]:
logging.set_verbosity_info()

####### LANCEMENT DE L'APPRENTISSAGE #######
trainer.train()

Une fois l'entraînement lancé, vous pouvez observer l'accumulation des points de contrôle dans le sous-dossier `checkpoints` du dossier de sortie.

À noter que si vous voulez vous servir éventuellement d'un point de contrôle du modèle, il faudrait suivre les étapes de fusion décrites ci-dessous, comme pour le modèle final.

#### Fusion des couches LoRA avec le modèle de base

Maintenant que nous avons entraîné des modules *adaptateurs* pour la réponse aux QCM, la dernière étape consiste en les réfusionner avec les poids fondamentaux du modèle.

Cela nous permet de ne pas rajouter du temps de calcul en inférence, contrairement à d'autres méthodes d'affinage par adaptateurs.

In [ ]:
merged_model = trainer.model.merge_and_unload()

In [ ]:
# sauvegarder le modèle
merged_model.save_pretrained(
    OUTPUT_DIR,
    safe_serialization=True,
    max_shard_size="1GB",
    save_embedding_layers=True
)
tokenizer.save_pretrained(OUTPUT_DIR)

### Évaluation du modèle entrainé (inférence)

In [ ]:
eval_ds = eval_ds.map(
    lambda x: tokenizer.apply_chat_template(
        x["prompt"],
        add_generation_prompt=True,
        enable_thinking=False,
        return_dict=True,
        padding=False  # fait par la fonction `mcqa_eval`
    ),
    desc="Tokenisation du jeu d'évaluation",
    remove_columns=["prompt"],
)

In [ ]:
eval_results = mcqa_eval(
    model=merged_model,
    tokenizer=tokenizer,
    dataset=eval_ds,
    batch_size=8,
    max_new_tokens=16
)
metrics_2print = "\n".join(f"{k.upper()} = {round(v * 100, 2)}" for k, v in eval_results["metrics"].items())
print(metrics_2print)